# Создание датасета с векторным представлением аккордов

## Импорт библиотек

In [ ]:
import json
from gensim.models import Word2Vec
from tqdm import tqdm
import pandas as pd

## Загружаем модель chord2vec

In [104]:
from gensim.models import Word2Vec
model = Word2Vec.load('chord2vecFiltData')

## Загружаем данные для обработки

### Загрузка всего набора данных

In [ ]:
# file = open("D:/muz/musicDrumBeats/Chordino.json")
# f = json.load(file)
# file = open("D:/muz/musicGitar/Chordino.json")
# f.update(json.load(file))
# file = open("D:/muz/musicPiano/Chordino.json")
# f = json.load(file)
# file = open("D:/muz/musicSaxophone/Chordino.json")
# f.update(json.load(file))
# file = open("D:/muz/musicYixInstrumental/Chordino.json")
# f.update(json.load(file))
# data = f

### Загрузка отфильтрованных данных

In [162]:
file = open("data_new_filtr078.json")
data = json.load(file)

## обработка последовательностей

### Чистим мелодии в которых аккордов меньше 30.

In [163]:
Fulldata = []
for i in data.keys():
    if(len(data[i]) >= 60):
        Fulldata.append(data[i])

### Разделям последовательности, на подпоследовательности длинной в 30 аккордов и 30 длительностей

In [188]:
n = 16
res = []
for data in Fulldata:
    res += [data[i:i+n] for i in range(0, len(data)-n+1, 2)]

### Кодируем аккорды векторами из word2vec модели(кроме последнего), и сохраняем в новую переменную df.

In [189]:
from tqdm import tqdm
target = []
df = []
for i in tqdm(res):
    target.append((i[14], i[15]))
    d = []
    for j in i[:14]:
        if j in model.wv.key_to_index.keys():
            d += list(model.wv[j])
        else:
            d.append(j)
    df.append(d)
#     df.append(i[:14])
    

100%|█████████████████████████████████████████████████████████████████████████| 29359/29359 [00:02<00:00, 11960.37it/s]


### Добавляем последние аккорды в df (столбцы target`ов)

In [190]:
for i in range(len(df)):
    df[i].append(target[i][0])
    df[i].append(target[i][1])

## Конвертируем полученный двумерный массив в сsv файл и сохраняем.

In [191]:
import pandas as pd

In [192]:
dataframe = pd.DataFrame(df)

In [193]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,699,700,701,702,703,704,705,706,707,708
0,-0.107933,0.078614,-0.017387,0.164854,0.069374,-0.097897,0.192984,0.279380,-0.258575,-0.243175,...,-0.203128,0.282263,0.193275,0.272685,-0.208255,-0.015003,0.041496,3.808073,G,4.365351
1,-0.150987,0.123017,-0.029146,0.312946,0.304819,-0.181564,0.532081,0.484460,-0.595567,-0.640385,...,-0.654997,0.705033,0.415587,0.649226,-0.597946,0.170293,0.268134,4.365351,Em,4.551111
2,-0.002452,0.087851,0.029628,0.216965,0.129866,-0.121652,0.228339,0.253636,-0.228816,-0.247632,...,-0.274381,0.358218,0.202193,0.576956,-0.523970,0.114061,0.083895,4.551111,Fmaj7,5.201270
3,-0.251732,0.190104,-0.048608,0.535927,0.297761,0.193033,0.351835,0.479407,-0.669690,-0.424112,...,-0.304939,0.598497,0.151609,0.474967,-0.472552,0.323630,0.070127,5.201270,Am,6.687347
4,-0.176404,0.225694,-0.049641,0.516211,0.282891,0.263275,0.190141,0.394450,-0.457227,-0.174002,...,-0.399139,0.485845,0.187864,0.604728,-0.503701,0.096247,0.149028,6.687347,Dm,2.693515


In [194]:
dataframe.shape

(29359, 709)

## Делим данные на train и test (соотношение 70/30), для обучения и оценивания моделей в будующем

In [ ]:
dataframe.rename(columns = {707 : 'Chord_target', 708: "Duration_target"}, inplace = True )
dataframe = dataframe.sample(frac=1).reset_index(drop=True)
Train = dataframe.loc[:int(len(dataframe)*0.7)]
Test = dataframe.loc[int(len(dataframe)*0.7):]

In [200]:
dataframe.head()

,0,1,2,3,4,5,6,7,8,9,...,699,700,701,702,703,704,705,706,Chord_target,Duration_target
0,-0.024530,0.204351,-0.055283,-0.020493,0.172326,-0.659137,0.368719,0.602145,-0.000755,-0.344135,...,-0.034031,0.560622,0.425777,0.022394,0.000091,-0.259136,-0.306072,1.671837,Cm,3.343673
1,0.026840,0.199608,0.019243,0.028806,-0.108556,-0.261151,-0.141279,0.344019,0.236750,0.134177,...,-0.335917,0.516764,0.266018,0.353768,-0.286438,0.170008,0.110870,16.625488,Bb6,1.671837
2,-0.223976,0.167835,-0.029918,0.547772,0.163190,0.257951,0.180622,0.410999,-0.545001,-0.229364,...,-0.561196,0.675920,0.464230,0.469520,-0.417726,0.051889,0.211325,4.086712,C,3.993832
3,-0.208511,-0.013439,-0.010159,0.211018,-0.129602,0.056328,0.025167,0.341366,-0.364835,-0.284785,...,-0.446547,0.615506,0.261426,0.488252,-0.411222,0.254105,0.165383,0.835918,Gm7,0.650159
4,0.050972,0.059549,0.014207,0.028896,0.211882,-0.435204,0.420850,0.363777,-0.155241,-0.391024,...,-0.561196,0.675920,0.464230,0.469520,-0.417726,0.051889,0.211325,3.715193,G7,0.928798


In [201]:
print(Test.shape, Train.shape)

(8808, 709) (20552, 709)


In [202]:
Test.to_csv('Chord2vec test filt.csv')

In [203]:
Train.to_csv('Chord2vec train filt.csv')